In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
# date of datas
date = "0913"

# arguments
cam_H, cam_W = 580, 890
wvls = np.arange(420, 670, 10)

n_illum = 318
n_patches = 24
n_channel = 3

patch_idx = np.arange(0, 24)
patch_size = 3

# dir
first_order_data_dir = './calibration/first_order_2023%s'%date

# center points for all patches
all_patches_center_pts = np.array([[210, 130], [324,142], [442,142], [558, 147], [680,152], [792,154],
[200,250], [326,253], [440,256], [560, 260], [676,261], [792,267],
[190,366], [318,372], [440,372], [558,376], [676,380], [794,379],
[190,490], [314,496], [436,491], [558,492], [678,495], [800,495]])

In [ ]:
first_order_data = []

exp_indx = 1

for i in range(318): 
    first_order_dir = os.path.join(first_order_data_dir, '%snm_crop/calibration00/capture_%04d.png')

    first_order_imgs = np.array([cv2.imread(first_order_dir%(wvls[w], i), -1)[:,:,::-1] for w in range(len(wvls))])
    first_order_data.append(first_order_imgs)
    
first_order_data = np.array(first_order_data) # 318, 25, 580, 890, 3

In [ ]:
first_order_data_transposed = first_order_data.transpose(2,3,0,1,4)

In [ ]:
# plot reflectance for each patches - real
# patches : 24개, wvl : 25 개
patches_rgb_real = np.zeros(shape = (n_illum, n_patches, len(wvls), n_channel))
patches_idx = all_patches_center_pts

cnst = 1
patch_size = 3

# max value defined for each illumination pattern 

for w in range(len(wvls)):
    for l in range(318):
            y_idx_start, y_idx_end = (patches_idx[:,1] - patch_size//2).astype(np.int32), (patches_idx[:,1] + patch_size//2).astype(np.int32)
            x_idx_start, x_idx_end = (patches_idx[:,0] - patch_size//2).astype(np.int32), (patches_idx[:,0] + patch_size//2).astype(np.int32)
            
            for p in range(n_patches):
                    y_idx, x_idx = patches_idx[p,1], patches_idx[p,0]
                    rgb_intensity = (first_order_data_transposed[y_idx_start[p]:y_idx_end[p], x_idx_start[p]:x_idx_end[p], l] / (65535)).reshape(-1, 3).mean(axis = 0)
                    patches_rgb_real[l, p, w] = rgb_intensity

In [ ]:
patches_rgb_real.shape # 318, 25, 580, 890, 3

In [ ]:
illum = np.arange(0, 318, 1)

start_idx = 0
end_idx = 318

for p in range(n_patches):
    plt.figure(figsize = (10,5))

    plt.plot(illum[start_idx: end_idx], patches_rgb_real[start_idx: end_idx, p, :, 0], label = 'real red', c = 'red'), plt.legend(), plt.ylim(0,1)
    plt.plot(illum[start_idx: end_idx], patches_rgb_real[start_idx: end_idx, p, :, 1], label = 'real green', c = 'green'), plt.legend(), plt.ylim(0,1)
    plt.plot(illum[start_idx: end_idx], patches_rgb_real[start_idx: end_idx, p, :, 2], label = 'real blue', c = 'blue'), plt.legend(), plt.ylim(0,1)

    plt.title('RGB intensity %s-th patch'%patch_idx[p])
    plt.xlabel('illumination index')
    plt.ylabel('rgb intensity')
    
    # plt.savefig('./%sms_%02d-th_patch.png'%(ex_time[exp_indx],patch_idx[p]))